<a href="https://colab.research.google.com/github/xinlei55555/PharmaHacks2023/blob/main/time_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#reading data

In [ ]:
import pandas as pd

aq_data = pd.read_json("/content/drive/MyDrive/ML/Hackathon (1)/Hackathon/AequitasData.json")
alpha_data = pd.read_json("/content/drive/MyDrive/ML/Hackathon (1)/Hackathon/AlphaData.json")
tsx_data = pd.read_json("/content/drive/MyDrive/ML/Hackathon (1)/Hackathon/TSXData.json")

total_df = pd.concat([aq_data, alpha_data, tsx_data])
total_df

,TimeStamp,TimeStampEpoch,Direction,OrderID,MessageType,Symbol,OrderPrice,Exchange
0,2023-01-06 09:28:00.013814157,2023-01-06 14:28:00.013814157,NBFToExchange,b8c529be-9283-11ed-ad3c-047c16291a22,NewOrderRequest,0455B,13.02,Aequitas
1,2023-01-06 09:28:00.013890960,2023-01-06 14:28:00.013890960,NBFToExchange,b8c529bf-9283-11ed-a725-047c16291a22,NewOrderRequest,0455B,13.14,Aequitas
2,2023-01-06 09:28:00.014031152,2023-01-06 14:28:00.014031152,NBFToExchange,b8c529c0-9283-11ed-bf41-047c16291a22,CancelRequest,0455B,NaN,Aequitas
3,2023-01-06 09:28:00.014086398,2023-01-06 14:28:00.014086398,NBFToExchange,b8c529c1-9283-11ed-8407-047c16291a22,CancelRequest,0455B,NaN,Aequitas
4,2023-01-06 09:28:00.014213645,2023-01-06 14:28:00.014213645,ExchangeToNBF,b8c529be-9283-11ed-ad3c-047c16291a22,NewOrderAcknowledged,0455B,13.02,Aequitas
...,...,...,...,...,...,...,...,...
212217,2023-01-06 09:31:59.995169747,2023-01-06 14:31:59.995169747,NBFToExchange,b9921f99-9283-11ed-a400-047c16291a22,NewOrderRequest,DC0OC,45.67,TSX
212218,2023-01-06 09:31:59.995210138,2023-01-06 14:31:59.995210138,NBFToExchange,b9921f83-9283-11ed-bcf1-047c16291a22,CancelRequest,DC0OC,NaN,TSX
212219,2023-01-06 09:31:59.995219075,2023-01-06 14:31:59.995219075,ExchangeToNBF,b9921f99-9283-11ed-a400-047c16291a22,NewOrderAcknowledged,DC0OC,45.67,TSX
212220,2023-01-06 09:31:59.995257550,2023-01-06 14:31:59.995257550,ExchangeToNBF,b9921f83-9283-11ed-bcf1-047c16291a22,CancelAcknowledged,DC0OC,NaN,TSX


In [ ]:
sample = total_df[total_df["OrderID"]=="b8c529be-9283-11ed-ad3c-047c16291a22"]
sample

,TimeStamp,TimeStampEpoch,Direction,OrderID,MessageType,Symbol,OrderPrice,Exchange
0,2023-01-06 09:28:00.013814157,2023-01-06 14:28:00.013814157,NBFToExchange,b8c529be-9283-11ed-ad3c-047c16291a22,NewOrderRequest,0455B,13.02,Aequitas
4,2023-01-06 09:28:00.014213645,2023-01-06 14:28:00.014213645,ExchangeToNBF,b8c529be-9283-11ed-ad3c-047c16291a22,NewOrderAcknowledged,0455B,13.02,Aequitas
172,2023-01-06 09:28:00.993798829,2023-01-06 14:28:00.993798829,NBFToExchange,b8c529be-9283-11ed-ad3c-047c16291a22,CancelRequest,0455B,NaN,Aequitas
174,2023-01-06 09:28:00.994174634,2023-01-06 14:28:00.994174634,ExchangeToNBF,b8c529be-9283-11ed-ad3c-047c16291a22,Cancelled,0455B,NaN,Aequitas


In [ ]:
print(type(sample.iloc[0]["TimeStamp"]))
sample.iloc[0]["TimeStamp"].timestamp()-sample.iloc[1]["TimeStamp"].timestamp()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


-0.0004000663757324219

#preparing new data

In [ ]:
import numpy as np
import datetime

In [ ]:
unique_ids = total_df["OrderID"].drop_duplicates()

req_time = []
cancel_time = []
r_exchanges = []
r_symbols = []
c_exchanges = []
c_symbols = []
o_anomaly_time = []
c_anomaly_time = []
# o_ids = []
# c_ids = []


for i in range(len(unique_ids)):
  id = unique_ids.iloc[i]

  transactions = total_df[total_df["OrderID"]==id]

  o_req = transactions[transactions["MessageType"]=="NewOrderRequest"]
  o_ack = transactions[transactions["MessageType"]=="NewOrderAcknowledged"]
  
  c_req = transactions[transactions["MessageType"]=="CancelRequest"]
  c_ack = transactions[transactions["MessageType"]=="CancelAcknowledged"]
  cancelled = transactions[transactions["MessageType"]=="Cancelled"]

  if (len(o_req)!=0 and len(o_ack)!=0):
    # print(o_ack.iloc[0]["TimeStamp"] == o_ack["TimeStamp"] )
    req_time.append((o_ack.iloc[0]["TimeStamp"].timestamp()-o_req.iloc[0]["TimeStamp"].timestamp())*1000)
    o_anomaly_time.append(np.squeeze(o_req["TimeStamp"].values))
    r_exchanges.append(str(np.squeeze(o_req["Exchange"].values)))
    r_symbols.append(str(np.squeeze(o_req["Symbol"].values)))
  else:
    req_time.append(np.nan)
    o_anomaly_time.append(np.nan)
    r_exchanges.append(np.nan)
    r_symbols.append(np.nan)


  if len(c_req)!=0:
    if len(c_ack)!=0:
      cancel_time.append((c_ack.iloc[0]["TimeStamp"].timestamp()-c_req.iloc[0]["TimeStamp"].timestamp())*1000)
    elif len(cancelled)!=0:
      cancel_time.append((cancelled.iloc[0]["TimeStamp"].timestamp()-c_req.iloc[0]["TimeStamp"].timestamp())*1000)
    else:
      cancel_time.append(np.nan)
    c_anomaly_time.append(np.squeeze(c_req["TimeStamp"].values))
    c_exchanges.append(str(np.squeeze(c_req["Exchange"].values)))
    c_symbols.append(str(np.squeeze(c_req["Symbol"].values)))
  else:
    cancel_time.append(np.nan)
    c_anomaly_time.append(np.nan)
    c_exchanges.append(np.nan)
    c_symbols.append(np.nan)

  # print(np.squeeze(o_req["Exchange"].values))
  # print(o_req["Exchange"])
  # print(o_req)
  
  



In [ ]:
#making the new datasets

order = pd.DataFrame()
order["id"] = unique_ids
order["time"] = o_anomaly_time
order["exchange"] = r_exchanges
order["symbol"] = r_symbols
order["time_difference"] = req_time

cancel = pd.DataFrame()
cancel["id"] = unique_ids
cancel["time"] = c_anomaly_time
cancel["exchange"] = c_exchanges
cancel["symbol"] = c_symbols
cancel["time_difference"] = cancel_time


In [ ]:
order = order.dropna()
order

,id,time,exchange,symbol,time_difference
0,b8c529be-9283-11ed-ad3c-047c16291a22,2023-01-06T09:28:00.013814157,Aequitas,0455B,0.400066
1,b8c529bf-9283-11ed-a725-047c16291a22,2023-01-06T09:28:00.013890960,Aequitas,0455B,0.390053
9,b8c529c3-9283-11ed-a276-047c16291a22,2023-01-06T09:28:00.023870827,Aequitas,OFQG6,0.394106
11,b8c529c5-9283-11ed-abe0-047c16291a22,2023-01-06T09:28:00.023991422,Aequitas,OFQG6,0.393867
16,b8c529c6-9283-11ed-b19a-047c16291a22,2023-01-06T09:28:00.032711042,Aequitas,7XHQS,0.408888
...,...,...,...,...,...
212197,b9921f95-9283-11ed-a89c-047c16291a22,2023-01-06T09:31:59.983188549,TSX,3PM7N,0.063181
212203,b9921f96-9283-11ed-a3af-047c16291a22,2023-01-06T09:31:59.985242391,TSX,OOOTO,0.052214
212208,b9921f97-9283-11ed-9d78-047c16291a22,2023-01-06T09:31:59.988659354,TSX,Z5HII,0.054121
212212,b9921f98-9283-11ed-842b-047c16291a22,2023-01-06T09:31:59.988701133,TSX,Z5HII,0.051975


In [ ]:
cancel = cancel.dropna()
cancel

,id,time,exchange,symbol,time_difference
0,b8c529be-9283-11ed-ad3c-047c16291a22,2023-01-06T09:28:00.993798829,Aequitas,0455B,0.375986
1,b8c529bf-9283-11ed-a725-047c16291a22,2023-01-06T09:29:02.597973368,Aequitas,0455B,0.382900
2,b8c529c0-9283-11ed-bf41-047c16291a22,2023-01-06T09:28:00.014031152,Aequitas,0455B,0.364065
3,b8c529c1-9283-11ed-8407-047c16291a22,2023-01-06T09:28:00.014086398,Aequitas,0455B,0.365019
8,b8c529c2-9283-11ed-9a22-047c16291a22,2023-01-06T09:28:00.023758872,Aequitas,OFQG6,0.369072
...,...,...,...,...,...
212122,b9921f86-9283-11ed-bb1d-047c16291a22,2023-01-06T09:31:59.974771987,TSX,0M3Q9,0.052929
212127,b9921f87-9283-11ed-8f1a-047c16291a22,2023-01-06T09:31:59.975621905,TSX,E4EMY,0.049114
212133,b9921f88-9283-11ed-8886-047c16291a22,2023-01-06T09:31:59.947365862,TSX,0M3Q9,0.047922
212138,b9921f89-9283-11ed-a909-047c16291a22,2023-01-06T09:31:59.948943351,TSX,0M3Q9,0.049114


In [ ]:
ord_data = order.drop(columns=["time", "id"])
can_data = cancel.drop(columns=["time", "id"])
type(ord_data["symbol"])

pandas.core.series.Series

In [ ]:
ord_data = pd.get_dummies(ord_data, columns=["symbol", "exchange"])
can_data = pd.get_dummies(can_data, columns=["symbol", "exchange"])
ord_data

,time_difference,symbol_0455B,symbol_05VGI,symbol_07FM5,symbol_07PKN,symbol_0M3Q9,symbol_1H2XW,symbol_1K95A,symbol_1NTNT,symbol_215JE,...,symbol_Y18QU,symbol_Y3SCQ,symbol_Z5HII,symbol_Z9T7U,symbol_ZASDH,symbol_ZHOXR,symbol_ZVLDO,exchange_Aequitas,exchange_Alpha,exchange_TSX
0,0.400066,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.390053,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0.394106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11,0.393867,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16,0.408888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212197,0.063181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
212203,0.052214,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
212208,0.054121,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
212212,0.051975,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
ord_data

,time_difference,symbol_0455B,symbol_05VGI,symbol_07FM5,symbol_07PKN,symbol_0M3Q9,symbol_1H2XW,symbol_1K95A,symbol_1NTNT,symbol_215JE,...,symbol_Y18QU,symbol_Y3SCQ,symbol_Z5HII,symbol_Z9T7U,symbol_ZASDH,symbol_ZHOXR,symbol_ZVLDO,exchange_Aequitas,exchange_Alpha,exchange_TSX
0,0.400066,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.390053,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0.394106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11,0.393867,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16,0.408888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212197,0.063181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
212203,0.052214,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
212208,0.054121,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
212212,0.051975,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


#Making the model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

##preparing train and test sets

In [ ]:
data_r = ord_data.drop(columns=["time_difference"])
labels_r = ord_data["time_difference"]

data_c = can_data.drop(columns=["time_difference"])
labels_c = can_data["time_difference"]

# data_c = cancel.iloc[:,:-1].values
# labels_c = cancel.iloc[:,-1].values
# data_c = data_c.drop(subset=["id", "time"])
# data_c = data_c.get_dummies()

In [ ]:
train_data_r, test_data_r, train_labels_r, test_labels_r = train_test_split(data_r, labels_r, test_size = 0.2, random_state = 42)
train_data_c, test_data_c, train_labels_c, test_labels_c = train_test_split(data_c, labels_c, test_size = 0.2, random_state = 42)
train_data_r.shape

(36719, 149)

In [ ]:
# #normalization

# min_r = tf.reduce_min(train_data_r)
# max_r = tf.reduce_max(train_data_r)

# train_data = (train_data_r - min_r)/(max_r - min_r)
# test_data = (test_data_r - min_r)/(max_r - min_r)

# train_data_r = tf.cast(train_data_r, dtype=tf.float32)
# test_data_r = tf.cast(test_data_r, dtype=tf.float32)

# min_c = tf.reduce_min(train_data_c)
# max_c = tf.reduce_max(train_data_c)

# train_data_c = (train_data_c - min_c)/(max_c - min_c)
# test_data_c = (test_data_c - min_c)/(max_c - min_c)

# train_data_c = tf.cast(train_data_c, dtype=tf.float32)
# test_data_c = tf.cast(test_data_c, dtype=tf.float32)


TypeError: ignored

In [ ]:
class detector(Model):
  def __init__(self, x):
    super(detector, self).__init__()
    self.encoder = tf.keras.Sequential([
                                        layers.Dense(10, activation="relu"),
                                        layers.Dense(10, activation='relu')
    ])
    self.decoder = tf.keras.Sequential([
                                        layers.Dense(10, activation='relu'),
                                        layers.Dense(x, activation='sigmoid')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [ ]:
autoencoder_r = detector(149)
autoencoder_r.compile(optimizer='adam', loss='mae')
autoencoder_r.fit(train_data_r, train_data_r, epochs = 20, batch_size=512, validation_data=(test_data_r, test_data_r))

Epoch 1/20
72/72 [==============================] - 1s 9ms/step - loss: 0.4736 - val_loss: 0.4045
Epoch 2/20
72/72 [==============================] - 1s 7ms/step - loss: 0.1892 - val_loss: 0.0282
Epoch 3/20
72/72 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0103
Epoch 4/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0093 - val_loss: 0.0088
Epoch 5/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0084 - val_loss: 0.0083
Epoch 6/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0082 - val_loss: 0.0081
Epoch 7/20
72/72 [==============================] - 0s 7ms/step - loss: 0.0080 - val_loss: 0.0081
Epoch 8/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0080 - val_loss: 0.0080
Epoch 9/20
72/72 [==============================] - 0s 7ms/step - loss: 0.0079 - val_loss: 0.0080
Epoch 10/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0079 - val_loss: 0.0080
Epoch 11/20
72/72 [

In [ ]:
reconstructed_r = autoencoder_r(train_data_r.get(0))

AttributeError: ignored

In [ ]:
reconstructed_r = autoencoder_r(train_data_r)
train_loss_r = losses.mae(reconstructed_r, train_data_r)
t_r = np.mean(train_loss_r) + np.std(train_loss_r)
t_r

39436              0             0             0             0             0   
113812             0             0             0             0             0   
159808             0             0             0             0             0   
94256              0             0             0             0             0   
174631             0             0             0             0             0   
...              ...           ...           ...           ...           ...   
36681              0             0             0             0             0   
206376             0             0             0             0             0   
173501             0             0             0             0             0   
3388               0             0             0             0             0   
61602              0             0             0             0             0   

        symbol_1H2XW  symbol_1K95A  symbol_1NTNT  symbol_215JE  symbol_2A523  \
39436              0             0     

AttributeError: ignored

In [ ]:
train_data_c.shape

(36656, 144)

In [ ]:
autoencoder_c = detector(144)
autoencoder_c.compile(optimizer='adam', loss='mae')
autoencoder_c.fit(train_data_c, train_data_c, epochs = 20, batch_size=512, validation_data=(test_data_c, test_data_c))

Epoch 1/20
72/72 [==============================] - 2s 11ms/step - loss: 0.4712 - val_loss: 0.3945
Epoch 2/20
72/72 [==============================] - 0s 6ms/step - loss: 0.1709 - val_loss: 0.0196
Epoch 3/20
72/72 [==============================] - 0s 7ms/step - loss: 0.0122 - val_loss: 0.0098
Epoch 4/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0092 - val_loss: 0.0089
Epoch 5/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0086 - val_loss: 0.0085
Epoch 6/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0084 - val_loss: 0.0083
Epoch 7/20
72/72 [==============================] - 1s 7ms/step - loss: 0.0082 - val_loss: 0.0083
Epoch 8/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 9/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 10/20
72/72 [==============================] - 0s 6ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 11/20
72/72 

In [ ]:
reconstructed_c = autoencoder_c(train_data_c)
train_loss_c = losses.mae(reconstructed_c, train_data_c)
t_c = np.mean(train_loss_c) + np.std(train_loss_c)

118773             0             0             0             0             0   
3759               0             0             0             0             0   
172613             0             0             0             0             0   
39077              0             0             0             0             0   
9189               0             0             0             0             0   
...              ...           ...           ...           ...           ...   
35715              0             0             0             0             0   
205987             0             0             0             0             0   
172723             0             0             0             0             0   
3370               0             0             0             0             0   
60210              0             0             0             0             0   

        symbol_1K95A  symbol_1NTNT  symbol_215JE  symbol_2A523  symbol_2E1Z7  \
118773             0             0     

AttributeError: ignored

In [ ]:
def prediction(model, data, threshold):
  rec = model(data)
  loss = losses.mae(rec, data)
  return tf.math.less(loss, threshold)

In [ ]:
pred_r = prediction(autoencoder_r, test_data_r, t_r)
print(pred)

NameError: ignored

In [ ]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.7-py3-none-any.whl size=181101 sha256=778a14d746c65e01537e514060ef6f80fe1f9f5abc6003f50266e94ed775c523
  Stored in directory: /root/.cache/pip/wheels/f7/e2/c1/1c7fd8b261e72411f6509afb429c84532e40ddcd96074473f4
Successfully built pyod


In [ ]:
from pyod.utils.data import generate_data

In [ ]:
from pyod.models.auto_encoder import AutoEncoder
atcdr_r = AutoEncoder(contamination=0.05, hidden_neurons =[2, 2])
atcdr_r.fit(train_data_r)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 149)               22350     
                                                                 
 dropout_4 (Dropout)         (None, 149)               0         
                                                                 
 dense_48 (Dense)            (None, 149)               22350     
                                                                 
 dropout_5 (Dropout)         (None, 149)               0         
                                                                 
 dense_49 (Dense)            (None, 2)                 300       
                                                                 
 dropout_6 (Dropout)         (None, 2)                 0         
                                                                 
 dense_50 (Dense)            (None, 2)               

AutoEncoder(batch_size=32, contamination=0.05, dropout_rate=0.2, epochs=100,
      hidden_activation='relu', hidden_neurons=[2, 2], l2_regularizer=0.1,
      loss=<function mean_squared_error at 0x7f882362f5e0>,
      optimizer='adam', output_activation='sigmoid', preprocessing=True,
      random_state=None, validation_size=0.1, verbose=1)

In [ ]:
# Training data
y_train_scores_r = atcdr_r.decision_function(train_data_r)
y_train_pred_r = atcdr_r.predict(train_data_r)

# Test data
y_test_scores_r = atcdr_r.decision_function(test_data_r)
y_test_pred = atcdr_r.predict(test_data_r) # outlier labels (0 or 1)

In [ ]:
atcdr_r = AutoEncoder(contamination=0.05, hidden_neurons =[2, 2])
atcdr_r.fit(train_data_r)